<a href="https://colab.research.google.com/github/Saadkhalid913/ML-Practice/blob/main/U_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torchvision.transforms as transforms 
import torch.nn.functional as F 
import torch.nn as nn
import numpy as np
import os 
import matplotlib.pyplot as plt 
from PIL import Image
import google.colab.drive as drive 
from torch.utils.data import Dataset, dataloader
import torch
import tensorflow as tf 
import tqdm
from skimage.filters import threshold_multiotsu
import seaborn as sns
from sklearn.cluster import KMeans


drive.mount("drive")

Mounted at drive


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
# !mkdir ../root/.kaggle
# !cp drive/MyDrive/kaggle/kaggle.json ../root/.kaggle/kaggle.json

In [8]:
import kaggle 

In [9]:
# !kaggle datasets download "dansbecker/cityscapes-image-pairs"


 97% 196M/202M [00:06<00:00, 30.6MB/s]
100% 202M/202M [00:06<00:00, 31.4MB/s]


In [ ]:
# !unzip cityscapes-image-pairs.zip

In [50]:
colors = [(255,0,0), (0,255,0), (0,0,255), (255,255,0), (255,0,255), 
              (0,255,255), (255,255,255), (200,50,0),(50,200,0), (50,0,200), 
              (200,200,50), (0,50,200), (0,200,50), (0,0,0)]
n_classes = 5
class UNETDataset(Dataset):

  def __init__(self):
    self.train_dir  = "cityscapes_data/train"
    self.image_names = next(os.walk(self.train_dir))[2][: 10000]
    self.number_of_images = len(self.image_names)
    self.km = None 

  @staticmethod
  def ColorSegmentedImage(img, n_classes):
    mask = np.zeros((3, 256,256), dtype = float)

    for i in range(n_classes):
      indices = (img[i] == 1)
      mask[0][indices] = colors[i][0]
      mask[1][indices] = colors[i][1]
      mask[2][indices] = colors[i][2]
    return mask

  @staticmethod
  def bin_image(mask):
      bins = np.arange(0,1,0.2, dtype=float)
      new_mask = np.digitize(mask, bins)
      return new_mask

  @staticmethod
  def getSegmentationArr(image, classes, width=256, height=256):
      seg_labels = np.zeros((classes, height, width))
      img = image[1, :, :,]

      for c in range(classes):
          seg_labels[c, :, :] = (img == c).astype(int)
      return seg_labels

  def train_kmeans(self, K=13):
    colors = []
    images = []
    for i in range(50):
      images.append(self[i][1])
    for image in images:
        colors.append(torch.reshape(image, (image.shape[1]*image.shape[2], 3)).numpy())
    colors = np.array(colors, dtype = float)

    print("Training K means on colors data of shape :", colors.shape)
    km = KMeans(K)
    km.fit(colors)
    # print("\nK-means clustering trained\n", km)
    # self.km = km 



  def __len__(self):
    return self.number_of_images

  def __getitem__(self, index):
      image_name = self.image_names[index]
      image_data = Image.open(f"{self.train_dir}/{image_name}")

      tensor_transform = transforms.ToTensor()
      image_data = tensor_transform(image_data)

      img = image_data[: , : ,  : 256 ]
      mask = image_data[: , : , 256 : ]

      # binned_mask = self.bin_image(mask)
      # segmented_mask = self.getSegmentationArr(binned_mask, classes = n_classes, width = 256, height = 256)


      # img = image_data[: , : ,  : 128 ]
      # mask = image_data[: , : , 128 : ]

      return img, mask



In [51]:
batch_size = 16
train_set = UNETDataset()
train_set.train_kmeans()
train_loader = dataloader.DataLoader(train_set, batch_size = batch_size, shuffle = False)

Training K means on colors data of shape : (50, 65536, 3)


ValueError: ignored

In [68]:
# def LayersToRGBImage(img):
#     colors = [(255,0,0), (0,255,0), (0,0,255), (255,255,0), (255,0,255), 
#               (0,255,255), (255,255,255), (200,50,0),(50,200,0), (50,0,200), 
#               (200,200,50), (0,50,200), (0,200,50), (0,0,0)]
    
#     nimg = np.zeros((3, img.shape[0], img.shape[1]))
#     for i in range(img.shape[0]):
#         c = img[i,:,:]
#         col = colors[i]
        
#         for color_channel in range(3):
#             nimg[color_channel, :,:] += col[color_channel]*c
#     nimg = nimg/255.0
#     return nimg



for (x,y, masks) in train_loader:

  mask = y[0]
  print(masks.shape)
  output = LayersToRGBImage(mask)
  print(output.shape)
  # k_means = KMeans(13)
  # tensor_colors = torch.tensor(colors, )


  # fig, ax = plt.subplots(nrows = x.shape[0], ncols = 2)
  # fig.set_figheight(128)
  # fig.set_figwidth(8)
  # for i in range(x.shape[0]):
  #   img = x[i]
  #   segmented = y[i]
  #   mask = masks[i]
  #   coloured_seg = train_set.ColorSegmentedImage(segmented, n_classes)
  #   ax[i,0].imshow(mask.permute(1,2,0))
  #   ax[i,1].imshow(coloured_seg.transpose(1,2,0))

  break

torch.Size([16, 3, 256, 256])


ValueError: ignored

In [11]:
# img = np.zeros((3, 256,256))
# img[[0], : 128, : 256] = np.full([1,128,256], fill_value = 255)
# img[[2], 128: , : 256] = np.full([1,128,256], fill_value = 255)
# # plt.imshow(img.transpose(1,2,0))

# img = torch.tensor(img)
# img = img.permute(1,2,0)
# img = img.reshape(-1, img.shape[2])
# # print(img[:10])
# # print(torch.unique(img, dim=0, return_counts = True))


In [15]:
class UNET(nn.Module):
  def __init__(self):

    super().__init__()

    self.learning_rate = 1e-4

    # batch norm 

    self.Batch_Norm = nn.BatchNorm2d(num_features=3)

    ## L1
    self.conv_1_1 = nn.Conv2d(3, 64, 3, 1,padding = "same")
    self.conv_1_2 = nn.Conv2d(64, 64, 3, 1, padding = "same")
    self.dropout_1 = nn.Dropout2d(0.2)

    self.max_pool_1 = nn.MaxPool2d(kernel_size = 2, return_indices = True)

    ## L2
    self.conv_2_1 = nn.Conv2d(64, 128, 3, 1,padding = "same")
    self.conv_2_2 = nn.Conv2d(128, 128, 3, 1, padding = "same")
    self.dropout_2 = nn.Dropout2d(0.2)

    self.max_pool_2 = nn.MaxPool2d(kernel_size = 2, return_indices = True)

    ## L3
    self.conv_3_1 = nn.Conv2d(128,256, 3, 1,padding = "same")
    self.conv_3_2 = nn.Conv2d(256, 256, 3, 1, padding = "same")
    self.dropout_3 = nn.Dropout2d(0.2)

    self.max_pool_3 = nn.MaxPool2d(kernel_size = 2, return_indices = True)

    ## L4
    self.conv_4_1 = nn.Conv2d(256, 512, 3, 1,padding = "same")
    self.conv_4_2 = nn.Conv2d(512, 512, 3, 1, padding = "same")
    self.dropout_4 = nn.Dropout2d(0.2)

    self.max_pool_4 = nn.MaxPool2d(kernel_size = 2, return_indices = True)

    ## L5
    self.conv_5_1 = nn.Conv2d(512, 1024, 1,padding = "same")
    self.conv_5_2 = nn.Conv2d(1024, 1024, 3, 1, padding = "same")
    self.dropout_5 = nn.Dropout2d(0.2)

    ##UP 1
    self.deconv1_1 = nn.ConvTranspose2d(1024, 512, 3, padding = 1, stride=2, output_padding = 1)
    self.deconv1_2 = nn.Conv2d(1024, 512, 3, 1, padding = "same")
    self.deconv1_3 = nn.Conv2d(512, 512, 3, 1, padding = "same")
    self.dropout_6 = nn.Dropout2d(0.2)


    ##UP 2
    self.deconv2_1 = nn.ConvTranspose2d(512, 256, 3, padding = 1, stride = 2, output_padding = 1)
    self.deconv2_2 = nn.Conv2d(512, 256, 3, 1, padding = "same")
    self.deconv2_3 = nn.Conv2d(256, 256, 3, 1, padding = "same")
    self.dropout_7 = nn.Dropout2d(0.2)


    ##UP 3
    self.deconv3_1 = nn.ConvTranspose2d(256, 128, 3, padding = 1, stride = 2, output_padding = 1)
    self.deconv3_2 = nn.Conv2d(256, 128, 3, 1, padding = "same")
    self.deconv3_3 = nn.Conv2d(128, 128, 3, 1, padding = "same")
    self.dropout_8 = nn.Dropout2d(0.2)


    ##UP 4
    self.deconv4_1 = nn.ConvTranspose2d(128, 64, 3, padding = 1, stride = 2, output_padding = 1)
    self.deconv4_2 = nn.Conv2d(128, 64, 3, 1, padding = 1)
    self.deconv4_3 = nn.Conv2d(64, 64, 3, 1, padding = "same")
    self.dropout_9 = nn.Dropout2d(0.2)


    ##UP 4
    self.deconv5_1 = nn.Conv2d(64, 64, 3, 1, padding = "same")
    self.deconv5_2 = nn.Conv2d(64, n_classes, 1, 1, padding = "same")







  def forward(self, x):
    x = self.Batch_Norm(x)
    L1_1 = self.conv_1_1(x)
    L1_1 = F.relu(L1_1)
    L1_2 = self.conv_1_2(L1_1)
    L1_2 = F.relu(L1_2)
    L1_2 = self.dropout_1(L1_2)


    L1_MAX, L1_indices = self.max_pool_1(L1_2)

    L2_1 = self.conv_2_1(L1_MAX)
    L2_1 = F.relu(L2_1)
    L2_2 = self.conv_2_2(L2_1)
    L2_2 = F.relu(L2_2)
    L2_2 = self.dropout_2(L2_2)

    L2_MAX, L2_indices = self.max_pool_2(L2_2)

    L3_1 = self.conv_3_1(L2_MAX)
    L3_1 = F.relu(L3_1)
    L3_2 = self.conv_3_2(L3_1)
    L3_2 = F.relu(L3_2)
    L3_2 = self.dropout_3(L3_2)

    L3_MAX, L3_indices = self.max_pool_3(L3_2)

    L4_1 = self.conv_4_1(L3_MAX)
    L4_1 = F.relu(L4_1)
    L4_2 = self.conv_4_2(L4_1)
    L4_2 = F.relu(L4_2)
    L4_2 = self.dropout_4(L4_2)

    L4_MAX, L4_indices = self.max_pool_4(L4_2)

    L5_1 = self.conv_5_1(L4_MAX)
    L5_1 = F.relu(L5_1)
    L5_2 = self.conv_5_2(L5_1)
    L5_2 = F.relu(L5_2)
    L5_2 = self.dropout_5(L5_2)

    ## Expansive Path

    D1_1 = self.deconv1_1(L5_2)
    skip_connection_1 = torch.cat([L4_2, D1_1], dim = 1)
    D1_2 = self.deconv1_2(skip_connection_1)
    D1_2 = F.relu(D1_2)
    D1_3 = self.deconv1_3(D1_2)
    D1_3 = F.relu(D1_3)
    D1_3 = self.dropout_6(D1_3)

    D2_1 = self.deconv2_1(D1_2)
    skip_connection_2 = torch.cat([L3_2, D2_1], dim = 1)
    D2_2 = self.deconv2_2(skip_connection_2)
    D2_2 = F.relu(D2_2)
    D2_3 = self.deconv2_3(D2_2)
    D2_3 = F.relu(D2_3)
    D2_3 = self.dropout_7(D2_3)

    D3_1 = self.deconv3_1(D2_2)
    skip_connection_3 = torch.cat([L2_2, D3_1], dim = 1)
    D3_2 = self.deconv3_2(skip_connection_3)
    D3_2 = F.relu(D3_2)
    D3_3 = self.deconv3_3(D3_2)
    D3_3 = F.relu(D3_3)
    D3_3 = self.dropout_8(D3_3)

    D4_1 = self.deconv4_1(D3_2)
    skip_connection_4 = torch.cat([L1_2, D4_1], dim = 1)
    D4_2 = self.deconv4_2(skip_connection_4)
    D4_2 = F.relu(D4_2)
    D4_3 = self.deconv4_3(D4_2)
    D4_3 = F.relu(D4_3)
    D4_3 = self.dropout_9(D4_3)

    D5_1 = self.deconv5_1(D4_2)
    D5_2 = self.deconv5_2(D5_1)
    D5_2 = torch.sigmoid(D5_2)



    return D5_2

In [16]:
NET = UNET()
NET = NET.to(device)
Optimizer = torch.optim.Adam(NET.parameters(), lr = NET.learning_rate)
Loss = nn.CrossEntropyLoss()

EPOCHS = 5
num_batches = 100 


In [17]:
NET.train()
for epoch in range(EPOCHS):
  Epoch_loss = 0
  group_loss = 0
  for i, (x,y) in enumerate(train_loader):

    if (i % 10 == 0):
      print("Batch Number: ", i + 1, " / ", len(train_loader), " LOSS: " + str(group_loss))
      group_loss = 0 
    

    x = x.to(device)
    y = y.to(device)

    output = NET(x)

    Optimizer.zero_grad()
    loss = Loss(output, y)
    loss.backward()
    Optimizer.step()

    Epoch_loss += loss.item()
    group_loss += loss.item()

  print(f"Epoch #{epoch + 1 } complete, LOSS: {Epoch_loss}")


Batch Number:  1  /  186  LOSS: 0
Batch Number:  11  /  186  LOSS: 22.69925632350578
Batch Number:  21  /  186  LOSS: 21.70202403792041
Batch Number:  31  /  186  LOSS: 18.403864656855717
Batch Number:  41  /  186  LOSS: 18.26359882470331
Batch Number:  51  /  186  LOSS: 18.445530709161517
Batch Number:  61  /  186  LOSS: 18.33069556389046
Batch Number:  71  /  186  LOSS: 18.32997782381915
Batch Number:  81  /  186  LOSS: 18.2944535176141
Batch Number:  91  /  186  LOSS: 18.326999100239846
Batch Number:  101  /  186  LOSS: 18.50777332039297
Batch Number:  111  /  186  LOSS: 18.332332352480762
Batch Number:  121  /  186  LOSS: 18.37590564949096
Batch Number:  131  /  186  LOSS: 18.29008197907706
Batch Number:  141  /  186  LOSS: 18.310634329643108
Batch Number:  151  /  186  LOSS: 18.35848689531781
Batch Number:  161  /  186  LOSS: 18.341594089205614
Batch Number:  171  /  186  LOSS: 18.382817037311725
Batch Number:  181  /  186  LOSS: 18.35896867499332
Epoch #1 complete, LOSS: 349.0820

KeyboardInterrupt: ignored

In [ ]:
class UNETDatasetTest(Dataset):

  def __init__(self):
    self.train_dir  = "cityscapes_data/val"
    self.image_names = next(os.walk(self.train_dir))[2]
    self.number_of_images = len(self.image_names)


  def __len__(self):
    return self.number_of_images

  def __getitem__(self, index):
      image_name = self.image_names[index]
      image_data = Image.open(f"{self.train_dir}/{image_name}")

      tensor_transform = transforms.ToTensor()
      resize_transform = transforms.Resize((64,128))

      image_data = tensor_transform(image_data)
      image_data = resize_transform(image_data)

      # img = image_data[: , : ,  : 256 ]
      # mask = image_data[: , : , 256 : ]

      img = image_data[: , : ,  : 64 ]
      mask = image_data[: , : , 64 : ]

      return img, mask

In [ ]:
# torch.save(NET.state_dict, "/saved_UNET_64x64")

In [ ]:
testset = UNETDatasetTest()
NET.eval()

x,y = testset[34]
x = x.unsqueeze(dim = 0)
output = NET(x.to(device))
img = output.squeeze(dim = 0).permute(1,2,0).detach().cpu().numpy()
# img = x.permute(1,2,0).detach().numpy()
mask = y.permute(1,2,0).detach().numpy()

pixels = np.concatenate((img, mask), axis = 1)
plt.imshow(pixels)
